# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО: Каюмов Эмиль Марселевич

Группа: 317

In [60]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import KFold
from sklearn.grid_search import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [4]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [5]:
data.shape

(32769, 10)

In [6]:
# доля положительных примеров
data.ACTION.mean()

0.94210992096188473

In [7]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [8]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния (учитите, что он должен возвращать вероятность — отношение объектов первого класса среди соседей к числу соседей). Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba).

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

Какая функция расстояния оказалась лучшей?

Для удобства дальнейшей работы переведём данные numpy-массивы.

In [9]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

Приготовим для каждого признака массивы $f$ и $p^2$. Для удобства для каждого признака построим словарь существующих в обучающей выборке категорий. Для скорости посчитаем логарифмы $f$ заранее (третья метрика начинает работать почти в три раза быстрее).

In [34]:
X_train = np.array(X_train, dtype='int64')

f = []
for i in range(X_train.shape[1]):
    dic = dict(zip(X_train[:, i], np.zeros(len(X_train[:, i]))))
    for key in dic.keys():
        dic[key] = float(np.sum(X_train[:, i] == key))
    f.append(dic)
   
p = []
for i in range(X_train.shape[1]):
    dic = dict(f[i])
    for key in dic.keys():
        dic[key] = dic[key] / X_train.shape[0] 
    p.append(dic)

p2 = []
for i in range(X_train.shape[1]):
    dic = dict(f[i])
    for key in dic.keys():
        dic[key] = (dic[key] * (dic[key] - 1)) / (X_train.shape[0] * (X_train.shape[0] - 1))
    p2.append(dic)

flog = []
for i in range(X_train.shape[1]):
    dic = {}
    for key in f[i].keys():
        dic[key] = np.log(f[i][key])
    flog.append(dic)

Реализуем метрики по формулам с семинара.

In [49]:
def metric_1(X, Y):
    return np.sum((X[:, np.newaxis] != Y[np.newaxis, :]), axis=2)

In [50]:
def metric_2(X, Y):
    result = np.zeros((X.shape[0], Y.shape[0]))
    pp2 = [np.array(p2[x].values()) for x in range(X.shape[1])]
    pp = [np.array(p[x].values()) for x in range(X.shape[1])]
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            for t in range(X.shape[1]):
                if x[t] == y[t] and x[t] in p2[t]:
                    result[i, j] += np.sum(pp2[t][pp[t] <= p[t].get(x[t], 0)])
    return result + metric_1(X, Y)

Реализуем третью метрику. Путём проб выяснено, что тройной цикл работает быстрее, чем если генерировать матрицу попарных расстояний через тройной вложенный генератор списков (с этой мыслью была реализована и вторая метрика).

In [51]:
def metric_3(X, Y):
    result = np.zeros((X.shape[0], Y.shape[0]))
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            for t in range(X.shape[1]):
                if x[t] != y[t]:
                    result[i, j] += flog[t].get(x[t], 0) * flog[t].get(y[t], 0)
    return result

Реализуем метод k ближайших соседей, возвращающий вероятность принадлежности объекта из теста классу 1 как долю объектов класса 1 среди k ближайших соседей. 

In [52]:
def knn(X_train, y_train, X_test, k, metric):
    return np.sum(y_train[metric(X_test, X_train).argsort(axis=1)[:, :k]], axis=1) / float(k)

Проверим первую метрику при k = 10:

In [53]:
y_predict = knn(X_train, y_train, X_test, 10, metric_1)
print 'auc =', roc_auc_score(y_test, y_predict)

auc = 0.83088009598


Проверим вторую метрику при k = 10:

In [54]:
y_predict = knn(X_train, y_train, X_test, 10, metric_2)
print 'auc =', roc_auc_score(y_test, y_predict)

auc = 0.832873632128


Проверим третью метрику при k = 10:

In [55]:
y_predict = knn(X_train, y_train, X_test, 10, metric_3)
print 'auc =', roc_auc_score(y_test, y_predict)

auc = 0.817053027738


Лучший результат показала вторая метрика (**0.833**). Однако результат первой метрики (**0.831**) незначительно уступила второй. Это можно объяснить тем, что значение второй метрики являлось суммой двух слагаемых, одно из которых являлось первой метрикой, а второе принимало заметно меньшие значения. Третья метрика показала чуть более низкий результат (**0.817**).

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

Реализуем функцию для поиска наилучшего параметра k.

In [56]:
def find_best_k(X_train, y_train, X_test, y_test, metric):
    K = range(1, 16)
    auc = np.zeros(15)
    index = metric(X_test, X_train).argsort(axis=1)
    for i, k in enumerate(K):
        y_predict = np.sum(y_train[index[:, :k]], axis=1) / float(k)
        auc[i] = roc_auc_score(y_test, y_predict)
    return np.argmax(auc) + 1, np.max(auc)

Найдём наилучший параметр k для первой метрики.

In [57]:
k, auc = find_best_k(X_train, y_train, X_test, y_test, metric_1)
print 'best k =', k
print 'auc =', auc

best k = 11
auc = 0.831201958708


Найдём наилучший параметр k для второй метрики.

In [58]:
k, auc = find_best_k(X_train, y_train, X_test, y_test, metric_2)
print 'best k =', k
print 'auc =', auc

best k = 7
auc = 0.836160998935


Найдём наилучший параметр k для третьей метрики.

In [59]:
k, auc = find_best_k(X_train, y_train, X_test, y_test, metric_3)
print 'best k =', k
print 'auc =', auc

best k = 11
auc = 0.817141176073


Наилучший результат (**0.836**) достигнут на второй метрике с k = 7. Для первой и третьей метрики лучшим оказался k = 11.

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

Реализуем функцию для подсчёта вещественных признаков по категориальным. Аргумент folds отвечает за фолдинг, а n_folds за количество фолдов.

In [22]:
def preprocess(X_train, X_test, y_train, folds=False, n_folds=1):
    
    X_train_new = np.zeros((X_train.shape[0], 0))
    X_test_new = np.zeros((X_test.shape[0], 0))
    
    for i in range(X_train.shape[1]):
        
        counts_test = np.zeros((X_test.shape[0], 1))
        clicks_test = np.zeros((X_test.shape[0], 1))
        for j in range(counts_test.shape[0]):
            counts_test[j] = np.sum(X_test[j, i] == X_train[:, i])
            clicks_test[j] = np.sum((X_test[j, i] == X_train[:, i]) & (y_train == 1))
        
        if folds == False:
            counts_train = np.zeros((X_train.shape[0], 1))
            clicks_train = np.zeros((X_train.shape[0], 1))
            for j in range(counts_train.shape[0]):
                counts_train[j] = np.sum(X_train[j, i] == X_train[:, i])
                clicks_train[j] = np.sum((X_train[j, i] == X_train[:, i]) & (y_train == 1))

        else: 
            fold_size = X_train.shape[0] // n_folds
            counts_train = np.zeros((0, 1))
            clicks_train = np.zeros((0, 1))
           
            for ind in range(n_folds):
        
                if ind == n_folds - 1 and fold_size * n_folds != X_train.shape[0]:
                    X_train_current = X_train[fold_size * ind:, i]
                    X_train_other = X_train[:fold_size * ind, i]
                    y_train_other = y_train[:fold_size * ind]
                else:
                    X_train_current = X_train[fold_size * ind : fold_size * (ind + 1), i]
                    X_train_other = np.hstack((X_train[:fold_size * ind, i], X_train[fold_size * (ind + 1):, i]))
                    y_train_other = np.hstack((y_train[:fold_size * ind], y_train[fold_size * (ind + 1):]))
            
                counts_train_part = np.zeros((X_train_current.shape[0], 1))
                clicks_train_part = np.zeros((X_train_current.shape[0], 1))
                
                for j in range(counts_train_part.shape[0]):
                    counts_train_part[j] = np.sum(X_train_current[j] == X_train_other)
                    clicks_train_part[j] = np.sum((X_train_current[j] == X_train_other) & (y_train_other == 1))
                
                counts_train = np.vstack((counts_train, counts_train_part))
                clicks_train = np.vstack((clicks_train, clicks_train_part))
                
            # масштабируем
            counts_train *= n_folds / (n_folds - 1)
            clicks_train *= n_folds / (n_folds - 1)
            
        X_train_new = np.hstack((X_train_new, counts_train, clicks_train, (clicks_train + 1) / (counts_train + 2)))
        X_test_new = np.hstack((X_test_new, counts_test, clicks_test, (clicks_test + 1) / (counts_test + 2)))
    
    return X_train_new, X_test_new

Преобразуем признаки (без фолдинга).

In [25]:
X_train_count, X_test_count = preprocess(X_train, X_test, y_train)
print X_train_count.shape, X_test_count.shape

(22938, 27) (9831, 27)


Преобразуем признаки (с фолдингом, n = 3).

In [23]:
X_train_count_fold, X_test_count_fold = preprocess(X_train, X_test, y_train, folds=True, n_folds=3)
print X_train_count_fold.shape, X_test_count_fold.shape

(22938, 27) (9831, 27)


Подберём параметр k для выборки без фолдинга.

In [133]:
cv = KFold(X_train_count.shape[0], n_folds=5, shuffle=True)
gs = GridSearchCV(KNeighborsClassifier(),
                  param_grid={'n_neighbors': [8, 10, 12, 14, 16, 18, 20]},
                  cv=cv,
                  scoring='roc_auc')
gs.fit(X_train_count, y_train)
print 'best score =', gs.best_score_
print gs.best_estimator_

best score = 0.771369947151
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=12, p=2, weights='uniform')


Запустим на тестовой выборке для подобранных параметров.

In [26]:
neigh = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_neighbors=12, 
                             p=2, weights='uniform')
neigh.fit(X_train_count, y_train)
y_predict = neigh.predict_proba(X_test_count)
print 'auc =', roc_auc_score(y_test, y_predict[:,1])

auc = 0.791252971524


Подберём параметр k для выборки с фолдингом.

In [134]:
cv = KFold(X_train_count_fold.shape[0], n_folds=5, shuffle=True)
gs = GridSearchCV(KNeighborsClassifier(),
                  param_grid={'n_neighbors': [8, 10, 12, 14, 16, 18, 20]},
                  cv=cv,
                  scoring='roc_auc')
gs.fit(X_train_count_fold, y_train)
print 'best score =', gs.best_score_
print gs.best_estimator_

best score = 0.744147377107
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=14, p=2, weights='uniform')


Запустим на тестовой выборке для подобранного параметра.

In [24]:
neigh = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_neighbors=14, 
                             p=2, weights='uniform')
neigh.fit(X_train_count_fold, y_train)
y_predict = neigh.predict_proba(X_test_count_fold)
print 'auc =', roc_auc_score(y_test, y_predict[:,1])

auc = 0.574670805056


Получаем, что kNN не переобучается на выборке без фолдинга. Результат даже падает при использовании фолдинга (**0.57** против **0.79** без фолдинга).

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

Добавим парные признаки.

In [159]:
X_train_pairwise = np.array(X_train)
X_test_pairwise = np.array(X_test)

for i in range(X_train.shape[1]):
    for j in range(X_train.shape[1]):
        if i != j:
            train = np.zeros((X_train.shape[0], 1))
            test = np.zeros((X_test.shape[0], 1))
            for ind in range(X_train.shape[0]):
                train[ind] = int(str(X_train[ind, i]) + str(X_train[ind, j]))
            for ind in range(X_test.shape[0]):
                test[ind] = int(str(X_test[ind, i]) + str(X_test[ind, j]))
            X_train_pairwise = np.hstack((X_train_pairwise, train))
            X_test_pairwise = np.hstack((X_test_pairwise, test))

Преобразуем выборку.

In [162]:
X_train_pairwise_count, X_test_pairwise_count = preprocess(X_train_pairwise, X_test_pairwise, y_train)
X_train_pairwise_count_fold, X_test_pairwise_count_fold = preprocess(X_train_pairwise, X_test_pairwise, y_train, 
                                                                     folds=True, n_folds=3)

Подберём значение k для выборки без фолдинга.

In [165]:
cv = KFold(X_train_pairwise_count.shape[0], n_folds=5, shuffle=True)
gs = GridSearchCV(KNeighborsClassifier(),
                  param_grid={'n_neighbors': [8, 10, 12, 14, 16, 18, 20]},
                  cv=cv,
                  scoring='roc_auc')
gs.fit(X_train_pairwise_count, y_train)
print 'best score =', gs.best_score_
print gs.best_estimator_

best score = 0.772132599652
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=14, p=2, weights='uniform')


Проверим на тестовой выборке для подобранных параметров.

In [330]:
neigh = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_neighbors=14, 
                             p=2, weights='uniform')
neigh.fit(X_train_pairwise_count, y_train)
y_predict = neigh.predict_proba(X_test_pairwise_count)
print 'auc =', roc_auc_score(y_test, y_predict[:,1])

auc = 0.798078042816


Подберём значение k для выборки с фолдингом.

In [166]:
cv = KFold(X_train_pairwise_count_fold.shape[0], n_folds=5, shuffle=True)
gs = GridSearchCV(KNeighborsClassifier(),
                  param_grid={'n_neighbors': [8, 10, 12, 14, 16, 18, 20]},
                  cv=cv,
                  scoring='roc_auc')
gs.fit(X_train_pairwise_count_fold, y_train)
print 'best score =', gs.best_score_
print gs.best_estimator_

best score = 0.758127156365
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=18, p=2, weights='uniform')


Проверим на тестовой выборке для подобранных параметров.

In [331]:
neigh = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_neighbors=18, 
                             p=2, weights='uniform')
neigh.fit(X_train_pairwise_count_fold, y_train)
y_predict = neigh.predict_proba(X_test_pairwise_count_fold)
print 'auc =', roc_auc_score(y_test, y_predict[:,1])

auc = 0.590832051993


Для выборки с парными признаками результаты аналогичны: kNN работает лучше на выборке без фолдинга (**0.8** против **0.59** с фолдингом). Парные признаки немного увеличили результат (**0.8** против **0.79**).

## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [326]:
cv = KFold(X_train_pairwise_count.shape[0], n_folds=5, shuffle=True)
gs = GridSearchCV(DecisionTreeClassifier(),
                  param_grid={'max_depth': [2, 3, 6, 9],
                              'min_samples_leaf': [2, 4, 6, 8, 10]},
                  cv=cv,
                  scoring='roc_auc')
gs.fit(X_train_pairwise_count, y_train)
print 'best score =', gs.best_score_
print gs.best_estimator_

best score = 0.999566130313
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=4,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')


In [327]:
dt = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3, max_features=None, max_leaf_nodes=None, 
                            min_samples_leaf=4, min_samples_split=2, min_weight_fraction_leaf=0.0, random_state=None, 
                            splitter='best')
dt.fit(X_train_pairwise_count, y_train)
y_predict = dt.predict_proba(X_test_pairwise_count)
print 'auc =', roc_auc_score(y_test, y_predict[:,1])

auc = 0.538328980728


Видим, как переобучились на выборке без фолдинга (**0.54** после 0.9996 на кросс-валидации).

#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

In [191]:
cv = KFold(X_train_pairwise_count.shape[0], n_folds=5, shuffle=True)
gs = GridSearchCV(RandomForestClassifier(),
                  param_grid={'n_estimators': [10, 20, 50, 100, 500, 1000]},
                  cv=cv,
                  scoring='roc_auc')
gs.fit(X_train_pairwise_count, y_train)
print 'best score =', gs.best_score_
print gs.best_estimator_

best score = 0.999980782368
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


Чем больше деревьев, тем лучше! Но куда ещё лучше?

Проверим на тестовой выборке.

In [192]:
rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=15, max_features='auto', 
                            max_leaf_nodes=None, min_samples_leaf=10, min_samples_split=2, 
                            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1, oob_score=False, 
                            random_state=None, verbose=0, warm_start=False)
rf.fit(X_train_pairwise_count, y_train)
y_predict = rf.predict_proba(X_test_pairwise_count)
print 'auc =', roc_auc_score(y_test, y_predict[:,1])

auc = 0.729228801808


Master of overfitting. 

Делаем вывод, что считать влоб clicks не очень хорошо. Хуже только забыть удалить y_train из data. 

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [209]:
cv = KFold(X_train_pairwise_count_fold.shape[0], n_folds=5, shuffle=True)
gs = GridSearchCV(RandomForestClassifier(),
                  param_grid={'n_estimators': [10, 50, 100, 1000]},
                  cv=cv,
                  scoring='roc_auc')
gs.fit(X_train_pairwise_count_fold, y_train)
print 'best score =', gs.best_score_
print gs.best_estimator_

best score = 0.868189142867
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=18, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


Запустим на тестовой выборке.

In [211]:
rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=18, max_features='auto', 
                            max_leaf_nodes=None, min_samples_leaf=10, min_samples_split=2, 
                            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1, oob_score=False, 
                            random_state=None, verbose=0, warm_start=False)
rf.fit(X_train_pairwise_count_fold, y_train)
y_predict = rf.predict_proba(X_test_pairwise_count_fold)
print 'auc =', roc_auc_score(y_test, y_predict[:,1])

auc = 0.857184152205


Видим, что на выборке с фолдингом результат отличается от кросс-валидации незначительно. То есть следует использовать фолдинг при подсчёте counts и clicks, иначе это может привести к переобучению. 

Наилучший auc (**0.857**) достигнут на случайном лесе с использованием преобразования категориальных признаков и парных признаках.
На kNN с помощью своей метрики достигнут результат **0.836**.